In [47]:
import pandas as pd
 
# File paths
draft_classes_path = 'data/Draft Classes 2014-2023.csv'
epm_data_path = 'data/EPM 2014-2024.csv'

# Load the datasets
draft_classes = pd.read_csv(draft_classes_path) 
epm_data = pd.read_csv(epm_data_path)

# Clean player names to ensure consistency between datasets
draft_classes['Player Name'] = draft_classes['Player Name'].str.strip().str.lower()
epm_data['Player Name'] = epm_data['Player Name'].str.strip().str.lower()

# Remove any columns that are entirely empty (all values NaN)
draft_classes = draft_classes.dropna(axis=1, how='all')


In [48]:
# Function to check eligibility based on minutes played
def is_eligible(minutes_series, required_years):
    return (minutes_series > 0).sum() >= required_years

# Function to extract player stats from the EPM dataset for each year
def get_player_stats(player_name, epm_data, year):
    player_stats = epm_data[(epm_data['Player Name'] == player_name) & (epm_data['Season'] == year)]
    return player_stats

In [49]:
import unicodedata
import re

# Function to normalize player names
def normalize_player_name(name):
    # Normalize non-English characters
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove periods and extra spaces
    name = re.sub(r'\s+', ' ', name).strip()  # Replace multiple spaces with a single space
    name = name.replace('.', '').strip()  # Remove periods
    name = re.sub(r'\s*(i{1,3}|iv|vi{0,3}|v?i{0,3})$', '', name)  # Remove trailing Roman numerals
    name = re.sub(r'\b(jr\.?|sr\.?|ii|iii|iv)\b', '', name, flags=re.IGNORECASE).strip()  # Remove suffixes

    return name

# Normalize player names in the Draft Classes dataset
draft_classes['Normalized Player Name'] = draft_classes['Player Name'].apply(normalize_player_name)

# Create a mapping from the EPM data
epm_data['Normalized Player Name'] = epm_data['Player Name'].apply(normalize_player_name)
name_mapping = dict(zip(epm_data['Normalized Player Name'], epm_data['Player Name']))

In [50]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
# Input data points (x = epm values at fixed percentiles,
# y = percentage of cap per minute at the same fixed percentiles)
# Output an exponential function that interpolates the data points

# Define the exponential function
def exponential_function(x, a, b):
    return a * np.exp(b * x)

def interpolate(x_data, y_data, plot=False):
    # Fit the data to the exponential function
    params, covariance = curve_fit(exponential_function, x_data, y_data)

    # Extract the parameters
    a_fit, b_fit = params

    if plot:
        # Generate points for plotting the fitted curve
        x_fit = np.linspace(min(x_data), max(x_data), 100)
        y_fit = exponential_function(x_fit, a_fit, b_fit)

        # Plot the original data points and the fitted curve
        plt.scatter(x_data, y_data, label="Data", color="red")
        plt.plot(x_fit, y_fit, label=f"Fitted Exponential: $y = {a_fit:.6f} e^{{{b_fit:.6f}x}}$", color="blue")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        plt.grid(True)
        plt.show()

    return lambda x: a_fit * np.exp(b_fit * x)

In [51]:
# Fit an exponential function to the 2024 data for EPM and salary per minute
# Data points
# EPM values at 10th, 50th, 70th, and 90th percentiles
x_data = np.array([-5.166, -1.691, -0.076, 2.4])
# percentage of cap per minute for the 10th, 50th, 70th, and 90th percentile salaries
# assumes each player plays every minute in the season
y_data = np.array([0.00000626, 0.00002744, 0.00006170, 0.00016])

epm_to_cap_units_per_minute = interpolate(x_data, y_data, plot=False)

def cumulative_cap_units(epm, minutes_played):
    return epm_to_cap_units_per_minute(epm) * minutes_played * 1000 # 1000 scaling factor so the numbers are intelligible

In [52]:
# Loop through each player in the entire draft class dataset and populate EPM and minutes for each year
for index, row in draft_classes.iterrows():
    player_name = row['Normalized Player Name']  # matching player names

    for year in range(2014, 2025):  # Loop through available years
        player_stats = get_player_stats(player_name, epm_data, year)
        
        minutes_col = f'Minutes_{year}'
        epm_col = f'EPM_{year}'
        # cumulative_epm_col = f'Cumulative_EPM_{year}'
        cumulative_cap_units_col = f'Cumulative_Cap_Units_{year}'
        
        # Initialize minutes and EPM with zeros
        draft_classes.loc[index, minutes_col] = 0  # Set default to 0
        draft_classes.loc[index, epm_col] = 0  # Set default to 0
        # draft_classes.loc[index, cumulative_epm_col] = float("-Inf")
        draft_classes.loc[index, cumulative_cap_units_col] = 0 # default to no cap units accrued

        # If player data exists for that year, extract and assign the minutes and EPM
        if not player_stats.empty:
            draft_classes.loc[index, minutes_col] = player_stats['Minutes'].values[0]  # Total minutes played
            draft_classes.loc[index, epm_col] = player_stats['EPM'].values[0]  # EPM value
            draft_classes.loc[index, cumulative_cap_units_col] = cumulative_cap_units(draft_classes.loc[index, epm_col], 
                                                             draft_classes.loc[index, minutes_col])

        # Calculate cumulative EPM (EPM * Minutes), only if minutes are greater than 0
        # if draft_classes.loc[index, minutes_col] > 0:
            # draft_classes.loc[index, cumulative_epm_col] = (draft_classes.loc[index, epm_col] * 
                                                            #  draft_classes.loc[index, minutes_col])

In [53]:
# # Calculate cumulative EPM for the entire career, first 5 years, and best 3 years
# for index, row in draft_classes.iterrows():
#     draft_year = row['Draft Year']  # matching player names

#     # Get all EPM and Minutes values for the player
#     epm_values = row[[f'Cumulative_EPM_{year}' for year in range(2014, 2025)]].values
#     minutes_values = row[[f'Minutes_{year}' for year in range(2014, 2025)]].values
    
#     # Calculate the total career cumulative EPM (including only years with minutes > 0)
#     total_cumulative_epm = sum(epm_values[minutes_values > 0])
#     draft_classes.loc[index, 'Total_Cumulative_EPM'] = total_cumulative_epm
    
#     # Calculate cumulative EPM for the first 5 years after the draft year
#     start_index = draft_year - 2014 + 1  # Find the start index for years after the draft
#     first_5_years_cumulative_epm = sum(epm_values[start_index:start_index + 5][minutes_values[start_index:start_index + 5] > 0])
#     draft_classes.loc[index, 'First_5_Years_Cumulative_EPM'] = first_5_years_cumulative_epm
    
#     # Store minutes played in the first 5 years
#     first_5_years_minutes = sum(minutes_values[start_index:start_index + 5][minutes_values[start_index:start_index + 5] > 0])
#     draft_classes.loc[index, 'First_5_Years_Minutes'] = first_5_years_minutes
    
    
#     # Calculate cumulative EPM for the best 3 years
#     # Filter out the years where the player has minutes > 0
#     valid_years = [year for year in range(2014, 2025) if row[f'Minutes_{year}'] > 0]

#     # If the player has fewer than 3 valid years, only use available years
#     if len(valid_years) < 3:
#         # If less than 3 valid years with minutes > 0, set values to 0
#         draft_classes.loc[index, 'Best_3_Years_Cumulative_EPM'] = 0
#         draft_classes.loc[index, 'Best_3_Years_Minutes'] = 0
#     else:
#         valid_epm_minutes = [(row[f'EPM_{year}'], row[f'Minutes_{year}']) for year in valid_years]

#         # Sort the list of tuples (EPM, Minutes) by EPM in descending order
#         sorted_epm_minutes = sorted(valid_epm_minutes, key=lambda x: x[0], reverse=True)

#         # Select the best 3 years (or fewer if not enough years)
#         best_3_years = sorted_epm_minutes[:3]

#         # Calculate cumulative EPM for the best 3 years
#         best_3_years_cumulative_epm = sum(epm * minutes for epm, minutes in best_3_years)
#         draft_classes.loc[index, 'Best_3_Years_Cumulative_EPM'] = best_3_years_cumulative_epm

#         # Store minutes played in the best 3 years
#         best_3_years_minutes = sum(minutes for _, minutes in best_3_years)
#         draft_classes.loc[index, 'Best_3_Years_Minutes'] = best_3_years_minutes


In [54]:
# # Create rankings within each draft class for total, first 5 years, and best 3 years
# for year in range(2014, 2025):
#     draft_classes['Rank_Cumulative_EPM'] = draft_classes.groupby('Draft Year')['Total_Cumulative_EPM'].rank(method='min', ascending=False)
#     draft_classes['Rank_First_5_Years_EPM'] = draft_classes.groupby('Draft Year')['First_5_Years_Cumulative_EPM'].rank(method='min', ascending=False)
#     draft_classes['Rank_Best_3_Years_EPM'] = draft_classes.groupby('Draft Year')['Best_3_Years_Cumulative_EPM'].rank(method='min', ascending=False)

# # Set rank to the lowest for players with zero minutes
# for index, row in draft_classes.iterrows():
#     if row['Total_Cumulative_EPM'] == 0:
#         draft_classes.loc[index, 'Rank_Cumulative_EPM'] = draft_classes['Rank_Cumulative_EPM'].max() + 1
#     if row['First_5_Years_Cumulative_EPM'] == 0:
#         draft_classes.loc[index, 'Rank_First_5_Years_EPM'] = draft_classes['Rank_First_5_Years_EPM'].max() + 1
#     if row['Best_3_Years_Cumulative_EPM'] == 0:
#         draft_classes.loc[index, 'Rank_Best_3_Years_EPM'] = draft_classes['Rank_Best_3_Years_EPM'].max() + 1

In [55]:
# # Calculate percentiles for ranks
# draft_classes['Percentile_Cumulative_EPM'] = draft_classes.apply(
#     lambda x: (1 - (x['Rank_Cumulative_EPM'] - 1) / draft_classes[draft_classes['Draft Year'] == x['Draft Year']].shape[0]) * 100,
#     axis=1
# )

# draft_classes['Percentile_First_5_Years_EPM'] = draft_classes.apply(
#     lambda x: (1 - (x['Rank_First_5_Years_EPM'] - 1) / draft_classes[draft_classes['Draft Year'] == x['Draft Year']].shape[0]) * 100,
#     axis=1
# )

# draft_classes['Percentile_Best_3_Years_EPM'] = draft_classes.apply(
#     lambda x: (1 - (x['Rank_Best_3_Years_EPM'] - 1) / draft_classes[draft_classes['Draft Year'] == x['Draft Year']].shape[0]) * 100,
#     axis=1
# )

In [56]:
# # Reorder columns to place cumulative EPM and percentiles together
# new_column_order = []
# for year in range(2014, 2025):
#     new_column_order.append(f'Minutes_{year}')
#     new_column_order.append(f'EPM_{year}')
#     new_column_order.append(f'Cumulative_EPM_{year}')

# # Get columns that are not part of the new order
# other_columns = draft_classes.columns.difference(new_column_order).tolist()
# final_column_order = other_columns + new_column_order + [
#     'Total_Cumulative_EPM', 
#     'Rank_Cumulative_EPM',
#     'First_5_Years_Cumulative_EPM',
#     'Rank_First_5_Years_EPM',
#     'Best_3_Years_Cumulative_EPM',
#     'Rank_Best_3_Years_EPM'
# ]

# # Reindex the DataFrame to the new order
# draft_classes = draft_classes[final_column_order]

In [ ]:
draft_classes

In [ ]:

# Save the modified dataframe to a new CSV file
output_file = 'data/Draft_Classes_with_EPM_all_years.csv'
draft_classes.to_csv(output_file, index=False)

output_file
